# Combining Python, SQL DBs and AWS

## Aim: Connect to your DB on AWS through python, and execute queries.  


# Using Python with MYSQL DB

In [7]:
# make sure we have the package installed
!pip install mysql-connector-python

In [36]:
## importing 'mysql.connector' 
import mysql.connector 

In [37]:
## Connecting to the database

## connecting to the database using 'connect()' method
## it takes 3 required parameters 'host', 'user', 'passwd'
cnx = mysql.connector.connect(
    host = "fis-db-instance.cepohieoey2w.us-east-1.rds.amazonaws.com",
    user = "admin",
    passwd = "H3cruejjcZ37qgl"
)

### Documentation:

https://dev.mysql.com/doc/connector-python/en/connector-python-connectargs.html

## Using a config File:

You do not want to make your credentials viewable to everyone who might see this file.  So instead of explicity stating your credentials, we want to import them from another file.  

In [38]:
import config

In [39]:
config.password

'H3cruejjcZ37qgl'

In [40]:
## Connecting to the database

## connecting to the database using 'connect()' method
## it takes 3 required parameters 'host', 'user', 'passwd'
cnx = mysql.connector .connect(
    host = config.host,
    user = config.user,
    passwd = config.password
)

print(cnx) # it will print a connection object if everything is fine


In [41]:
cursor = cnx.cursor()

## SQL Commands: Data Definition Language Commands (DDL)
This section will discuss the commands through which you can define your database. The commands are as follows:

- CREATE
- DROP
- TRUNCATE
- ALTER
- BACKUP DATABASE

### CREATE

This statement is used to create a table or a database.



#### The ‘CREATE DATABASE’ Statement:
As the name suggests, this statement is used to create a database.

**Syntax**

`CREATE DATABASE DatabaseName;`

**Example**

`CREATE DATABASE Employee;`


#### The **‘CREATE TABLE’** Statement

This statement is used to create a table.

**Syntax**

`CREATE TABLE TableName (
Column1 datatype,
Column2 datatype,
Column3 datatype,
....
ColumnN datatype
);`

**Example**

`CREATE TABLE Employee_Info
(
EmployeeID int,
EmployeeName varchar(255),
Emergency ContactName varchar(255),
PhoneNumber int,
Address varchar(255),
City varchar(255),
Country varchar(255)
);`

You can also create a table using another table. Refer the below sytax and example:

#### The ‘CREATE TABLE AS’ Statement
**Syntax**

`CREATE TABLE NewTableName AS
SELECT Column1, column2,..., ColumnN
FROM ExistingTableName
WHERE ....;`

**Example**

`CREATE TABLE ExampleTable AS
SELECT EmployeeName, PhoneNumber
FROM Employee_Info;`

[MYSQL Data Types](http://www.mysqltutorial.org/mysql-data-types.aspx)

### DROP
This statement is used to drop an existing table or a database.

#### The ‘DROP DATABASE’ Statement
This statement is used to drop an existing database. When you use this statement, complete information present in the database will be lost.

**Syntax**

`DROP DATABASE DatabaseName;`

**Example**

`DROP DATABASE Employee;`

#### The ‘DROP TABLE’ Statement
This statement is used to drop an existing table. When you use this statement, complete information present in the table will be lost.

**Syntax**

`DROP TABLE TableName;`

**Example**

`DROP Table Employee_Info;`

### TRUNCATE
This command is used to delete the information present in the table but does not delete the table. So, once you use this command, your information will be lost, but not the table.

**Syntax**

`TRUNCATE TABLE TableName;`

**Example**

`TRUNCATE Table Employee_Info;`

### The ‘ALTER TABLE’ Statement
This statement is used to add, delete, modify columns in an existing table.

#### The ‘ALTER TABLE’ Statement with ADD/DROP COLUMN
You can use the ALTER TABLE statement with ADD/DROP Column command according to your need. If you wish to add a column, then you will use the ADD command, and if you wish to delete a column, then you will use the DROP COLUMN command.

**Syntax**
`ALTER TABLE TableName
ADD ColumnName Datatype;`

`ALTER TABLE TableName
DROP COLUMN ColumnName;`

**Example**

`ALTER TABLE Employee_Info
ADD BloodGroup varchar(255);`
 
`ALTER TABLE Employee_Info
DROP COLUMN BloodGroup ;`

#### The ‘ALTER TABLE’ Statement with ALTER/MODIFY COLUMN
This statement is used to change the datatype of an existing column in a table.

**Syntax**

`ALTER TABLE TableName
ALTER COLUMN ColumnName Datatype;`

**Example**
 
`ALTER TABLE Employee_Info
ADD DOB year;
ALTER TABLE Employee_Info
ALTER DOB date;`

### BACKUP DATABASE
This statement is used to create a full backup of an existing database.

**Syntax**
`BACKUP DATABASE DatabaseName
TO DISK = 'filepath';`

**Example**

`BACKUP DATABASE Employee
TO DISK = 'C:UsersSahitiDesktop';`


## SQL Commands: Constraints Used In Database
Constraints are used in a database to specify the rules for data in a table. The following are the different types of constraints:

- NOT NULL
- UNIQUE
- CHECK
- DEFAULT
- INDEX


### NOT NULL
This constraint ensures that a column cannot have a NULL value.

**Example**

 
`CREATE TABLE Employee_Info
(
EmployeeID int NOT NULL,
EmployeeName varchar(255) NOT NULL,
Emergency ContactName varchar(255),
PhoneNumber int NOT NULL,
Address varchar(255),
City varchar(255),
Country varchar(255)
);`
 
NOT NULL on ALTER TABLE
 
`ALTER TABLE Employee_Info
MODIFY PhoneNumber int NOT NULL;`

### UNIQUE
This constraint ensures that all the values in a column are unique.

**Example**

UNIQUE on Create Table
 
`CREATE TABLE Employee_Info
(
EmployeeID int NOT NULL UNIQUE,
EmployeeName varchar(255) NOT NULL,
Emergency ContactName varchar(255),
PhoneNumber int NOT NULL,
Address varchar(255),
City varchar(255),
Country varchar(255)
);`
 
UNIQUE on Multiple Columns
 
`CREATE TABLE Employee_Info
(
EmployeeID int NOT NULL,
EmployeeName varchar(255) NOT NULL,
Emergency ContactName varchar(255),
PhoneNumber int NOT NULL,
Address varchar(255),
City varchar(255),
Country varchar(255),
CONSTRAINT UC_Employee_Info UNIQUE(Employee_ID, PhoneNumber)
);`
 
UNIQUE on ALTER TABLE
 
`ALTER TABLE Employee_Info
ADD UNIQUE (Employee_ID);`
 
To drop a UNIQUE constraint
 
`ALTER TABLE  Employee_Info
DROP CONSTRAINT UC_Employee_Info;`

## Creating DB

In [186]:
from mysql.connector import errorcode

In [187]:
db_name = 'employees'

In [188]:
def create_database(cursor, database):
    try:
        cursor.execute(
            "CREATE DATABASE {} DEFAULT CHARACTER SET 'utf8'".format(database))
    except mysql.connector.Error as err:
        print("Failed creating database: {}".format(err))
        exit(1)

try:
    cursor.execute("USE {}".format(db_name))
except mysql.connector.Error as err:
    print("Database {} does not exists.".format(db_name))
    if err.errno == errorcode.ER_BAD_DB_ERROR:
        create_database(cursor, db_name)
        print("Database {} created successfully.".format(db_name))
        cnx.database = db_name
    else:
        print(err)
        exit(1)

Database employees does not exists.
2013 (HY000): Lost connection to MySQL server during query


### Creating tables

In [45]:
DB_NAME = 'employees'

TABLES = {}
TABLES['employees'] = (
    "CREATE TABLE employees ("
    "  emp_no int(11) NOT NULL AUTO_INCREMENT,"
    "  birth_date date NOT NULL,"
    "  first_name varchar(14) NOT NULL,"
    "  last_name varchar(16) NOT NULL,"
    "  gender enum('M','F') NOT NULL,"
    "  hire_date date NOT NULL,"
    "  PRIMARY KEY (emp_no)"
    ") ENGINE=InnoDB")

TABLES['departments'] = (
    "CREATE TABLE departments ("
    "  dept_no char(4) NOT NULL,"
    "  dept_name varchar(40) NOT NULL,"
    "  PRIMARY KEY (dept_no), UNIQUE KEY dept_name (dept_name)"
    ") ENGINE=InnoDB")

TABLES['salaries'] = (
    "CREATE TABLE salaries ("
    "  emp_no int(11) NOT NULL,"
    "  salary int(11) NOT NULL,"
    "  from_date date NOT NULL,"
    "  to_date date NOT NULL,"
    "  PRIMARY KEY (emp_no,from_date), KEY emp_no (emp_no),"
    "  CONSTRAINT salaries_ibfk_1 FOREIGN KEY (emp_no) "
    "     REFERENCES employees (emp_no) ON DELETE CASCADE"
    ") ENGINE=InnoDB")

TABLES['dept_emp'] = (
    "CREATE TABLE dept_emp ("
    "  emp_no int(11) NOT NULL,"
    "  dept_no char(4) NOT NULL,"
    "  from_date date NOT NULL,"
    "  to_date date NOT NULL,"
    "  PRIMARY KEY (emp_no,dept_no), KEY emp_no (emp_no),"
    "  KEY dept_no (dept_no),"
    "  CONSTRAINT dept_emp_ibfk_1 FOREIGN KEY (emp_no) "
    "     REFERENCES employees (emp_no) ON DELETE CASCADE,"
    "  CONSTRAINT dept_emp_ibfk_2 FOREIGN KEY (dept_no) "
    "     REFERENCES departments (dept_no) ON DELETE CASCADE"
    ") ENGINE=InnoDB")

TABLES['dept_manager'] = (
    "  CREATE TABLE dept_manager ("
    "  dept_no char(4) NOT NULL,"
    "  emp_no int(11) NOT NULL,"
    "  from_date date NOT NULL,"
    "  to_date date NOT NULL,"
    "  PRIMARY KEY (emp_no,dept_no),"
    "  KEY emp_no (emp_no),"
    "  KEY dept_no (dept_no),"
    "  CONSTRAINT dept_manager_ibfk_1 FOREIGN KEY (emp_no) "
    "     REFERENCES employees (emp_no) ON DELETE CASCADE,"
    "  CONSTRAINT dept_manager_ibfk_2 FOREIGN KEY (dept_no) "
    "     REFERENCES departments (dept_no) ON DELETE CASCADE"
    ") ENGINE=InnoDB")

TABLES['titles'] = (
    "CREATE TABLE titles ("
    "  emp_no int(11) NOT NULL,"
    "  title varchar(50) NOT NULL,"
    "  from_date date NOT NULL,"
    "  to_date date DEFAULT NULL,"
    "  PRIMARY KEY (emp_no,title,from_date), KEY emp_no (emp_no),"
    "  CONSTRAINT titles_ibfk_1 FOREIGN KEY (emp_no)"
    "     REFERENCES employees (emp_no) ON DELETE CASCADE"
    ") ENGINE=InnoDB")

In [34]:
TABLES['employees']

"CREATE TABLE employees (  emp_no int(11) NOT NULL AUTO_INCREMENT,  birth_date date NOT NULL,  first_name varchar(14) NOT NULL,  last_name varchar(16) NOT NULL,  gender enum('M','F') NOT NULL,  hire_date date NOT NULL,  PRIMARY KEY (emp_no)) ENGINE=InnoDB"

In [46]:
for table_name in TABLES:
    table_description = TABLES[table_name]
    try:
        print("Creating table {}: ".format(table_name), end='')
        cursor.execute(table_description)
    except mysql.connector.Error as err:
        if err.errno == errorcode.ER_TABLE_EXISTS_ERROR:
            print("already exists.")
        else:
            print(err.msg)
    else:
        print("OK")

cursor.close()
cnx.close()

Creating table employees: already exists.
Creating table departments: already exists.
Creating table salaries: already exists.
Creating table dept_emp: already exists.
Creating table dept_manager: already exists.
Creating table titles: already exists.


## Inserting Data

In [61]:
#since we closed the connection, we need to reinstatiate it
cnx = mysql.connector.connect(
    host = config.host,
    user = config.user,
    passwd = config.password,
    database = DB_NAME
)
cursor = cnx.cursor()

In [62]:
# cursor = create_cnx()

In [63]:
from datetime import date, datetime, timedelta

tomorrow = datetime.now().date() + timedelta(days=1)

add_employee = ("INSERT INTO employees "
               "(first_name, last_name, hire_date, gender, birth_date) "
               "VALUES (%s, %s, %s, %s, %s)")
add_salary = ("INSERT INTO salaries "
              "(emp_no, salary, from_date, to_date) "
              "VALUES (%(emp_no)s, %(salary)s, %(from_date)s, %(to_date)s)")

data_employee = ('Geert', 'Vanderkelen', tomorrow, 'M', date(1977, 6, 14))



In [64]:
data_employee

('Geert',
 'Vanderkelen',
 datetime.date(2019, 10, 24),
 'M',
 datetime.date(1977, 6, 14))

In [65]:
# Insert new employee
cursor.execute(add_employee, data_employee)
emp_no = cursor.lastrowid

In [66]:
emp_no

3

In [67]:
# Insert salary information
data_salary = {
  'emp_no': emp_no,
  'salary': 50000,
  'from_date': tomorrow,
  'to_date': date(9999, 1, 1),
}
cursor.execute(add_salary, data_salary)

# Make sure data is committed to the database
cnx.commit()



## Insert Many

cursor.executemany(operation, seq_of_params)

In [68]:
cnx = mysql.connector.connect(
    host = config.host,
    user = config.user,
    passwd = config.password,
    database = DB_NAME
)
cursor = cnx.cursor()

In [69]:
data = [
  ('Jane', date(2005, 2, 12)),
  ('Joe', date(2006, 5, 23)),
  ('John', date(2010, 10, 3)),
]
stmt = "INSERT INTO employees (first_name, hire_date) VALUES (%s, %s)"
cursor.executemany(stmt, data)

In [70]:
cnx.commit()

In [71]:
cursor.close()
cnx.close()

## Querying the DB

In [164]:
cnx = mysql.connector.connect(
    host = config.host,
    user = config.user,
    passwd = config.password,
    database = DB_NAME
)
cursor = cnx.cursor()

In [76]:

query = ("SELECT first_name, last_name, hire_date FROM employees "
         "WHERE hire_date BETWEEN %s AND %s")

hire_start = date(2005, 1, 1)
hire_end = date(2010, 12, 31)

cursor.execute(query, (hire_start, hire_end))

In [77]:
for (first_name, last_name, hire_date) in cursor:
  print("{}, {} was hired on {:%d %b %Y}".format(
    last_name, first_name, hire_date))

cursor.close()
cnx.close()

, Jane was hired on 12 Feb 2005
, Joe was hired on 23 May 2006
, John was hired on 03 Oct 2010


## Applied: Working with our student data

### Import our student data

In [152]:
import json

In [153]:
f=open('students.json','r')
data=json.load(f)

### Create a connection and cursor to AWS db

In [154]:
DB_NAME

'employees'

In [167]:
cnx = mysql.connector.connect(
    host = config.host,
    user = config.user,
    passwd = config.password,
    database = DB_NAME
)
cursor = cnx.cursor()

### Create a table for our student info

In [142]:
data[3]

{'name': 'Clare Blessen',
 'dob': '1994-08-10',
 'birthplace': 'Chicago, IL',
 'siblings': 1,
 'years_in_nyc': 2,
 'favorite_food': 'cheese'}

In [150]:
create_query = """
                CREATE TABLE students 
                (
                    student_id int(7) NOT NULL UNIQUE AUTO_INCREMENT,
                    name varchar(20) NOT NULL,
                    dob date NOT NULL,
                    birthplace varchar(50) NOT NULL,
                    siblings int(2) NOT NULL,
                    years_in_nyc float(3) NOT NULL,
                    favorite_food varchar(50) NOT NULL
                )
                """

# drop_table = """DROP Table students;"""


In [151]:

# cursor.execute(drop_table)
cursor.execute(create_query)

### Insert the student data into the table

In [145]:
data[0]

{'name': 'Sean Abu Wilson',
 'dob': '1985-02-06',
 'birthplace': 'Birmingham, AL',
 'siblings': 2,
 'years_in_nyc': 8.4,
 'favoritefood': 'oatmeal raisin cookies'}

In [157]:
students_list = []

for student in data:
    if 'favoritefood' in student.keys():
        student["favorite_food"] = student["favoritefood"]
#     if 'DOB' in student.keys():
#         student['birthddate'] = student['DOB']
#     if 'Birthplace' in student.keys():
#         studdent['birthplace'] = student.pop('Birthplace')
        
    student_tuple = (student['name'],
                     student['dob'],
                     student['siblings'],
                     student['birthplace'],
                     student['years_in_nyc'],
                     student['favorite_food'])
    students_list.append(student_tuple)

students_list

[('Sean Abu Wilson',
  '1985-02-06',
  2,
  'Birmingham, AL',
  8.4,
  'oatmeal raisin cookies'),
 ('Rebecca Chi-May Weng',
  '1996-05-17',
  0,
  'New York, NY',
  18,
  'Taiwanese beef noodle soup'),
 ('Charlie Schlinkert', '1994-06-11', 2, 'Darien, Ct', 7.2, 'Hamburgers'),
 ('Clare Blessen', '1994-08-10', 1, 'Chicago, IL', 2, 'cheese'),
 ('Dave Bletsch', '1982-04-29', 2, 'New Hyde Park, NY', 0.6, 'buffalo wings'),
 ('Eddie Pognon', '1994-04-26', 4, 'Newark, New Jersey', 22, 'pasta'),
 ('Findlay Bowditch', '1992-08-22', 2, 'White Plains, NY', 1, 'BBQ'),
 ('Gabriel Seemann', '1991-02-09', 3, 'San Diego, CA', 2.2, 'Falafel'),
 ('Garrett Keyes', '1993-09-20', 0, 'New York City, NY', 3.3, 'lamb vindaloo'),
 ('Jake Cohen', '1995-02-12', 1, 'Philadelphia, PA', 1.4, 'pizza'),
 ('JP Rinfret',
  '1990-08-02',
  2,
  'Manhasset, NY',
  5.6,
  'breaded chicken cutlets'),
 ('Kenny Miyasato', '1988-07-13', 0, 'Lima, Peru', 0, 'Pizza'),
 ('Kyle McNicoll', '1994-02-11', 1, 'NYC', 24, 'salmon sushi'

In [158]:
insert_query = """
INSERT INTO students (name, dob, siblings, birthplace, years_in_nyc, favorite_food) VALUES(%s, %s, %s, %s, %s, %s)
"""

In [159]:
cursor.executemany(insert_query, students_list)
cnx.commit()

In [130]:
#
keys_list = []
for student in data:
    print(student['name'])
    keys = student.keys()
    for word in keys:
        if word not in keys_list:
            keys_list.append(word)
print (keys_list)

Sean Abu Wilson
Rebecca Chi-May Weng
Charlie Schlinkert
Clare Blessen
Dave Bletsch
Eddie Pognon
Findlay Bowditch
Gabriel Seemann
Garrett Keyes
Jake Cohen
JP Rinfret
Kenny Miyasato
Kyle McNicoll
Marco Sanchez-Ayala
Matthew R Finch
Melissa Munz
Michael Eby
Ravi Charan
Regina Ryan
Samuel Diaz
Sasha Epelbaum
Stephanie Bourdeau
Wesley Willis
Zoe Zhu
Christopher Shaw
['name', 'dob', 'birthplace', 'siblings', 'years_in_nyc', 'favoritefood', 'favorite_food']


In [160]:
student = data[0]

In [161]:
student

{'name': 'Sean Abu Wilson',
 'dob': '1985-02-06',
 'birthplace': 'Birmingham, AL',
 'siblings': 2,
 'years_in_nyc': 8.4,
 'favoritefood': 'oatmeal raisin cookies',
 'favorite_food': 'oatmeal raisin cookies'}

### Write queries to answer the following questions:

**Questions**
- Which student is the youngest?
- Which student's birthday is closest to the cohort's graduation date?
- Which student has the most siblings?
- How many students are only children?
- Which 3 students have lived in NYC the shortest amount of time?
- How many students are native New Yorkers?
- What are the most popular foods among the class?

[Utilizing the date data type in MySQL](http://www.mysqltutorial.org/mysql-date/)

In [175]:
# Which student is the youngest?
import pandas as pd
cnx = mysql.connector.connect(
    host = config.host,
    user = config.user,
    passwd = config.password,
    database = DB_NAME
)
cursor = cnx.cursor()

cursor.execute("""
SELECT
    *
FROM
    employees.students s
ORDER BY
    s.dob DESC
LIMIT
    1
;
""")
df = pd.DataFrame(cursor.fetchall())
df.columns = [x[0] for x in cursor.description]
df

,student_id,name,dob,birthplace,siblings,years_in_nyc,favorite_food
0,21,Sasha Epelbaum,1996-10-05,"NYC, NY",1,19.0,nyc bagels


In [184]:
# Which student's birthday is closest to the cohort's graduation date?
import pandas as pd
cnx = mysql.connector.connect(
    host = config.host,
    user = config.user,
    passwd = config.password,
    database = DB_NAME
)
cursor = cnx.cursor()

cursor.execute("""
SELECT
    *,
    ABS(DAYOFYEAR(s.dob) -  DAYOFYEAR('2020/01/24')) AS days_from_graduation
FROM
    employees.students s
ORDER BY
    days_from_graduation
LIMIT
    1
;
""")
df = pd.DataFrame(cursor.fetchall())
df.columns = [x[0] for x in cursor.description]
df

,student_id,name,dob,birthplace,siblings,years_in_nyc,favorite_food,days_from_graduation
0,1,Sean Abu Wilson,1985-02-06,"Birmingham, AL",2,8.4,oatmeal raisin cookies,13


In [185]:
# Which student has the most siblings?
import pandas as pd
cnx = mysql.connector.connect(
    host = config.host,
    user = config.user,
    passwd = config.password,
    database = DB_NAME
)
cursor = cnx.cursor()

cursor.execute("""
SELECT
    *
FROM
    employees.students s
WHERE
    s.siblings = (SELECT
                    MAX(s.siblings)
                    FROM
                    employees.students s)
;
""")
df = pd.DataFrame(cursor.fetchall())
df.columns = [x[0] for x in cursor.description]
df

,student_id,name,dob,birthplace,siblings,years_in_nyc,favorite_food
0,6,Eddie Pognon,1994-04-26,"Newark, New Jersey",4,22.0,pasta
1,23,Wesley Willis,1992-06-04,"Detroit, MI",4,7.0,banana fritters


In [181]:
# How many students are only children?
import pandas as pd
cnx = mysql.connector.connect(
    host = config.host,
    user = config.user,
    passwd = config.password,
    database = DB_NAME
)
cursor = cnx.cursor()

cursor.execute("""
SELECT
    COUNT(s.siblings)
FROM
    employees.students s
WHERE
    s.siblings = 0
;
""")
df = pd.DataFrame(cursor.fetchall())
df.columns = [x[0] for x in cursor.description]
df

,COUNT(s.siblings)
0,5


In [182]:
# Which 3 students have lived in NYC the shortest amount of time?

import pandas as pd
cnx = mysql.connector.connect(
    host = config.host,
    user = config.user,
    passwd = config.password,
    database = DB_NAME
)
cursor = cnx.cursor()

cursor.execute("""
SELECT
    *
FROM
    employees.students s
ORDER BY
    s.years_in_nyc ASC
LIMIT
    3
;
""")
df = pd.DataFrame(cursor.fetchall())
df.columns = [x[0] for x in cursor.description]
df

,student_id,name,dob,birthplace,siblings,years_in_nyc,favorite_food
0,12,Kenny Miyasato,1988-07-13,"Lima, Peru",0,0.0,Pizza
1,5,Dave Bletsch,1982-04-29,"New Hyde Park, NY",2,0.6,buffalo wings
2,7,Findlay Bowditch,1992-08-22,"White Plains, NY",2,1.0,BBQ


In [183]:
# How many students are native New Yorkers?
import pandas as pd
cnx = mysql.connector.connect(
    host = config.host,
    user = config.user,
    passwd = config.password,
    database = DB_NAME
)
cursor = cnx.cursor()

cursor.execute("""
SELECT
    *
FROM
    employees.students s
WHERE
    s.birthplace LIKE '%NY%' AND 
    s.birthplace NOT LIKE '%New Hyde Park%' AND
    s.birthplace NOT LIKE '%White Plains%' AND
    s.birthplace NOT LIKE '%Manhasset%'
;
""")
df = pd.DataFrame(cursor.fetchall())
df.columns = [x[0] for x in cursor.description]
df

,student_id,name,dob,birthplace,siblings,years_in_nyc,favorite_food
0,2,Rebecca Chi-May Weng,1996-05-17,"New York, NY",0,18.0,Taiwanese beef noodle soup
1,9,Garrett Keyes,1993-09-20,"New York City, NY",0,3.3,lamb vindaloo
2,13,Kyle McNicoll,1994-02-11,NYC,1,24.0,salmon sushi
3,20,Samuel Diaz,1987-09-21,"Harlem, NY",2,32.0,Grilled Steak with Rice and Broccoli
4,21,Sasha Epelbaum,1996-10-05,"NYC, NY",1,19.0,nyc bagels
5,22,Stephanie Bourdeau,1993-09-30,"Brooklyn, NY",2,26.1,NYC pizza


In [ ]:
# What are the most popular foods among the class?
import pandas as pd
cnx = mysql.connector.connect(
    host = config.host,
    user = config.user,
    passwd = config.password,
    database = DB_NAME
)
cursor = cnx.cursor()

cursor.execute("""

""")
df = pd.DataFrame(cursor.fetchall())
df.columns = [x[0] for x in cursor.description]
df